# Get Wiki page and use bs4 to read in the table data. Create pandas datafram from table data

In [1]:
import requests
import bs4
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
b = bs4.BeautifulSoup(page.text)
t = b.find('table', class_='wikitable sortable')

# create dataframe for the table
Postcode = []
Borough = []
Neighbourhood = []
for tr in t.find_all('tr')[1:]:
    Postcode.append(tr.find('td').text.strip())
    Borough.append(tr.find('td').find_next('td').text.strip())
    Neighbourhood.append(tr.find('td').find_next('td').find_next('td').text.strip())

import pandas as pd
d = pd.DataFrame()
d['Postcode'] = Postcode
d['Borough'] = Borough
d['Neighbourhood'] = Neighbourhood
d=d[d['Borough']!='Not assigned']
d[d['Neighbourhood']=='Not assigned']['Neighbourhood']=d['Borough']
d.set_index(['Postcode','Borough'], inplace=True)
d
Mailcode = d.groupby(level=['Postcode','Borough'], sort=False).agg(','.join)
Mailcode.reset_index(inplace=True)

C:\Users\Jia\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Print out .shape for submission
# 1.This is for the first submission 

In [2]:
Mailcode.shape

(103, 3)

In [3]:
import io
csv = requests.get('http://cocl.us/Geospatial_data').content.decode('utf-8')
l = pd.read_csv(io.StringIO(csv), delimiter=',')
Mailcode_l = Mailcode.merge(l,how='left',left_on='Postcode', right_on="Postal Code")
Mailcode_l.drop('Postal Code', axis=1, inplace=True)
Mailcode_l.shape

(103, 5)

# 2. This is for the 2nd submission

In [4]:
Mailcode_l.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


# 3.1. Map out the Neighbourhoods of Toronto

In [5]:
import folium
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Mailcode_l['Latitude'], \
Mailcode_l['Longitude'], Mailcode_l['Borough'], Mailcode_l['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto


In [6]:
CLIENT_ID = 'J1JNSELXZ11ESAK5NVYDKRGJWHUVHX0QW3VHEOM53L03L1LS' # your Foursquare ID
CLIENT_SECRET = 'WEENNVISZHZRKUL2HJJWJIIXW0WAUEUG3MDYUOVBWJCRGDTS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# 3.2. Explore Neighborhoods in Toronto

In [7]:
toronto_venues = getNearbyVenues(names=Mailcode_l['Neighbourhood'],
                                   latitudes=Mailcode_l['Latitude'],
                                   longitudes=Mailcode_l['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Not assigned
Islington Avenue
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design E

In [8]:
print(toronto_venues.shape)
toronto_venues.head()

(2244, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


# 3.3. Analyze Each Neighbourhood in Toronto

In [9]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()


(2244, 280)


,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(101, 280)


,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.01
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00


In [11]:
import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Hotel,Asian Restaurant,American Restaurant,Breakfast Spot,Gym
1,Agincourt,Lounge,Skating Rink,Breakfast Spot,Chinese Restaurant,Sandwich Place,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Pharmacy,Sandwich Place,Beer Store,Pizza Place,Coffee Shop,Fast Food Restaurant,Electronics Store,Eastern European Restaurant
4,"Alderwood,Long Branch",Pizza Place,Gym,Pharmacy,Sandwich Place,Pub,Pool,Skating Rink,Coffee Shop,Construction & Landscaping,Empanada Restaurant


# 3.4. Cluster Neighborhoods in Toronto to 5 clusters and append other fields

In [12]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10] )

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Mailcode_l

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.merge(neighborhoods_venues_sorted, how='inner',\
                                      left_on='Neighbourhood',right_on='Neighborhood')

toronto_merged.head() # check the last columns!

[1 1 2 1 1 1 1 1 1 1]


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Parkwoods,Fast Food Restaurant,Food & Drink Shop,Park,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Victoria Village,Financial or Legal Service,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,1,"Harbourfront,Regent Park",Coffee Shop,Bakery,Café,Pub,Park,Theater,Restaurant,Breakfast Spot,Mexican Restaurant,Cosmetics Shop
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,1,"Lawrence Heights,Lawrence Manor",Clothing Store,Women's Store,Shoe Store,Furniture / Home Store,Arts & Crafts Store,Miscellaneous Shop,Coffee Shop,Event Space,Vietnamese Restaurant,Boutique
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494,1,Not assigned,Coffee Shop,Park,Gym,Diner,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café


let's map out the clusters

In [13]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], 
                                  toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], 
                                  toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 3.5. Examine Clusters

cluster# 0

In [14]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, \
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,North York,0,"Silver Hills,York Mills",Cafeteria,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner


cluster# 1

In [15]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, \
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Victoria Village,Financial or Legal Service,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant
2,Downtown Toronto,1,"Harbourfront,Regent Park",Coffee Shop,Bakery,Café,Pub,Park,Theater,Restaurant,Breakfast Spot,Mexican Restaurant,Cosmetics Shop
3,North York,1,"Lawrence Heights,Lawrence Manor",Clothing Store,Women's Store,Shoe Store,Furniture / Home Store,Arts & Crafts Store,Miscellaneous Shop,Coffee Shop,Event Space,Vietnamese Restaurant,Boutique
4,Queen's Park,1,Not assigned,Coffee Shop,Park,Gym,Diner,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café
6,North York,1,Don Mills North,Gym / Fitness Center,Basketball Court,Caribbean Restaurant,Japanese Restaurant,Café,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Event Space,Dive Bar
7,East York,1,"Woodbine Gardens,Parkview Hill",Fast Food Restaurant,Pizza Place,Gym / Fitness Center,Bank,Pet Store,Pharmacy,Intersection,Athletics & Sports,Gastropub,Donut Shop
8,Downtown Toronto,1,"Ryerson,Garden District",Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Theater,Fast Food Restaurant,Restaurant,Japanese Restaurant,Bookstore
9,North York,1,Glencairn,Bakery,Pub,Italian Restaurant,Japanese Restaurant,Women's Store,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
10,Etobicoke,1,"Cloverdale,Islington,Martin Grove,Princess Gar...",Bank,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
11,Scarborough,1,"Highland Creek,Rouge Hill,Port Union",Bar,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner


cluster# 2

In [16]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, \
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Parkwoods,Fast Food Restaurant,Food & Drink Shop,Park,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
20,York,2,Caledonia-Fairbanks,Park,Women's Store,Market,Fast Food Restaurant,Concert Hall,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Comic Shop
34,East York,2,East Toronto,Park,Pizza Place,Convenience Store,Coffee Shop,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
39,North York,2,"CFB Toronto,Downsview East",Park,Airport,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
51,North York,2,"Newtonbrook,Willowdale",Park,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
60,Central Toronto,2,Lawrence Park,Park,Swim School,Bus Line,Dim Sum Restaurant,Concert Hall,Construction & Landscaping,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
63,York,2,Weston,Park,Convenience Store,Women's Store,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
65,North York,2,York Mills West,Park,Convenience Store,Bank,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
67,Central Toronto,2,"Forest Hill North,Forest Hill West",Trail,Park,Sushi Restaurant,Jewelry Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
76,Etobicoke,2,"Kingsview Village,Martin Grove Gardens,Richvie...",Park,Bus Line,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner


cluster# 3

In [17]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, \
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,3,"Rouge,Malvern",Fast Food Restaurant,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


cluster# 4

In [18]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, \
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,4,Roselawn,Garden,Women's Store,Dim Sum Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
